In [1]:
import pandas as pd
import numpy  as np
from openai import OpenAI
import time

In [2]:
client = OpenAI(
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
    api_key='a0BIj000001UF1bMAG'
)


def summarise_text(text):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": f"The following texts summarise the reponses. Each response is delimited by \n. Can you briefly summarise some common key points in all the responses (use bullet points and each point is explained within 20 words)? The texts are shown below:\n\n{text}"}
        ]
    )

    # Correctly access the message content from the response
    message_content = completion.choices[0].message.content.strip().lower()
    
    return message_content


In [3]:
# read the excel file
df = pd.read_excel('241124_survey.xlsx')

In [4]:
# print the first few rows of the table
print(df.head())

   ID  sessionID  p_label_num     chosen_feedback       vote_feedback  \
0   1          1            1  Timebased feedback  Timebased feedback   
1   2          1            2  Timebased feedback  Timebased feedback   
2   3          1            3  Timebased feedback  Timebased feedback   
3   4          1            4  Timebased feedback  Timebased feedback   
4   5          1            5  Timebased feedback  Timebased feedback   

                               vote_feedback_explain         vote_change  \
0  it was clearer how many tokens members contrib...  Timebased feedback   
1  It's clear to show the amount of our team's co...  Timebased feedback   
2  I could gather a better understanding of what ...  Timebased feedback   
3  Option 1 is much clearer regarding the contrib...  Timebased feedback   
4  because I could decide what amount of point i ...  Timebased feedback   

                                 vote_change_explain  
0            it was clearer to see the team this 

In [5]:
check_D_E = df['chosen_feedback'] == df['vote_feedback']
print(df[~check_D_E])

      ID  sessionID  p_label_num     chosen_feedback vote_feedback  \
222  274         11            4  Milestone feedback  Not remember   

    vote_feedback_explain vote_change vote_change_explain  
222                   NaN           .                 NaN  


We know that:
1. there is one participant who did not remember the type of feedback. The information is shown above
2. all the remaining participants remembered their choice correctly.

In [6]:
number_of_participants = df.shape[0]
print(f"The number of participants is {number_of_participants}")

The number of participants is 267


In [7]:
number_of_timebased = df['chosen_feedback'].value_counts()['Timebased feedback']
print(f"The number of participants who chose 'Timebased feedback' is {number_of_timebased}")

The number of participants who chose 'Timebased feedback' is 199


In [8]:
df_timebased = df[(df['chosen_feedback'] == 'Timebased feedback')]
vote_feedback_explain_timebased = '\n'.join(df_timebased['vote_feedback_explain'].astype(str))
print(f"Some key reasons of participants choosing timebased feedback are: \n\n {summarise_text(vote_feedback_explain_timebased)}")

Some key reasons of participants choosing timebased feedback are: 

 the common key points across the responses are:

- **clarity of contributions**: participants value seeing the exact token or effort units contributed by everyone for informed decision-making.

- **decision making**: the detailed contribution information helps participants make strategic decisions on their own contributions.

- **trust and cooperation**: transparency in contributions affects trust levels and encourages or discourages cooperation among team members.

- **real-time information**: having up-to-date information on contributions facilitates timely decision-making adjustments.

- **risk management**: exact contribution data allows participants to better assess risks and optimize their strategies for maximizing gains.

- **preference for transparency**: detailed and transparent contribution feedback is preferred to vague milestones for clear strategizing.


In [9]:
number_of_milestone = df['chosen_feedback'].value_counts()['Milestone feedback']
print(f"The number of participants who chose 'Milestone feedback' is {number_of_milestone}")

The number of participants who chose 'Milestone feedback' is 68


In [10]:
df_milestone = df[(df['chosen_feedback'] == 'Milestone feedback')]
vote_feedback_explain_milestone = '\n'.join(df_milestone['vote_feedback_explain'].astype(str))
print(f"Some key reasons of participants choosing milestone feedback are: \n\n {summarise_text(vote_feedback_explain_milestone)}")

Some key reasons of participants choosing milestone feedback are: 

 the responses share several common key points:

- **preference for ambiguity**: many participants preferred not knowing exact contributions, enabling strategic decisions and profit maximization.
  
- **reduced pressure**: vague information lowered pressure to contribute more than peers.
  
- **influence on strategy**: concealing contributions allows participants to keep strategies private.
  
- **encouragement of contributions**: milestone-based feedback motivated team contributions without exact contribution disclosure.
  
- **manipulation of perception**: participants believed vagueness might convince others to contribute more.
  
- **risk and profit maximization**: uncertainty was leveraged to increase personal gains by exploiting potential team behavior.
  
- **reduced accountability**: ambiguity diminished personal accountability and judgment from others.


In [11]:
timebased_1 = df['chosen_feedback'] == 'Timebased feedback'
df_temp = df[timebased_1]
number_of_timebased_timebased = df_temp['vote_change'].value_counts()['Timebased feedback']
print(f"The number of participants who chose 'Timebased feedback' originally and stayed in 'Timebased feedback' is {number_of_timebased_timebased}")
df_timebased_timebased = df[(df['chosen_feedback'] == 'Timebased feedback') & (df['vote_change'] == 'Timebased feedback')]
vote_change_explain_timebased_timebased = '\n'.join(df_timebased_timebased['vote_change_explain'].astype(str))
print(f"Some key reasons of participants choosing timebased feedback and staying the same are: \n\n {summarise_text(vote_change_explain_timebased_timebased)}")

The number of participants who chose 'Timebased feedback' originally and stayed in 'Timebased feedback' is 182
Some key reasons of participants choosing timebased feedback and staying the same are: 

 based on the responses, here are some common key points:

- **desire for transparency**: respondents valued clear visibility into team members' contributions for better decision-making.

- **preference for detailed information**: many preferred having exact numbers to get a full understanding of group dynamics.

- **better decision-making**: detailed feedback was seen as helpful in making more informed and rational choices.

- **influence of others' contributions**: understanding other participants' efforts allowed for strategy adjustments and cooperation.

- **avoiding uncertainty**: respondents preferred clear and precise data to reduce uncertainty and feel more secure in their decisions.

- **team dynamics**: knowledge of exact contributions was important to gauge teamwork and willingn

In [12]:
number_of_timebased_milestone = df_temp['vote_change'].value_counts()['Milestone feedback']
print(f"The number of participants who chose 'Timebased feedback' originally and changed to 'Milestone feedback' is {number_of_timebased_milestone}")
df_timebased_milestone = df[(df['chosen_feedback'] == 'Timebased feedback') & (df['vote_change'] == 'Milestone feedback')]
vote_change_explain_timebased_milestone = '\n'.join(df_timebased_milestone['vote_change_explain'].astype(str))
print(f"Some key reasons of participants choosing timebased feedback but changing to milestone feedback are: \n\n {summarise_text(vote_change_explain_timebased_milestone)}")

The number of participants who chose 'Timebased feedback' originally and changed to 'Milestone feedback' is 17
Some key reasons of participants choosing timebased feedback but changing to milestone feedback are: 

 - **team contribution motivation:** awareness of collective goals can inspire team members to contribute more.
- **transparency vs. ambiguity:** ambiguity in feedback can motivate equal contributions but might also enable manipulation.
- **experimenting feedback options:** participants expressed interest in trying different feedback methods to observe changes in team dynamics.
- **human behavior contrasts with ai:** participants noted differences between human and ai motivations.
- **milestone tracking:** participants considered milestone visibility as a factor affecting contribution levels.
- **self-reflection on contribution levels:** some participants admitted to being selfish or reconsidered their contribution levels if repeated.


In [13]:
milestone_1 = df['chosen_feedback'] == 'Milestone feedback'
df_temp = df[milestone_1]
number_of_milestone_milestone = df_temp['vote_change'].value_counts()['Milestone feedback']
print(f"The number of participants who chose 'Milestone feedback' originally and stayed in 'Milestone feedback' is {number_of_milestone_milestone}")
df_milestone_milestone = df[(df['chosen_feedback'] == 'Milestone feedback') & (df['vote_change'] == 'Milestone feedback')]
vote_change_explain_milestone_milestone = '\n'.join(df_milestone_milestone['vote_change_explain'].astype(str))
print(f"Some key reasons of participants choosing milestone feedback and staying the same are: \n\n {summarise_text(vote_change_explain_milestone_milestone)}")

The number of participants who chose 'Milestone feedback' originally and stayed in 'Milestone feedback' is 54
Some key reasons of participants choosing milestone feedback and staying the same are: 

 here are some common key points from the responses:

- **preference for milestones**: many prefer milestones over exact numbers for clarity and easier decision-making.
  
- **strategic contribution**: individuals are inclined to contribute less when exact totals are unknown, optimizing personal benefits.
  
- **trust and risk management**: milestones enhance trust and strategic risk-taking among team members.
  
- **prisoner's dilemma reference**: decisions influenced by assumptions about others' self-serving actions.
  
- **transparency vs. curiosity**: less transparency fosters curiosity and possibly greater effort to meet milestones.
  
- **frustration with selfishness**: some cite teammates being selfish as a reason for selecting options that obscure personal contributions.
  
- **cons

In [14]:
number_of_milestone_timebased = df_temp['vote_change'].value_counts()['Timebased feedback']
print(f"The number of participants who chose 'Milestone feedback' originally and changed to 'Timebased feedback' is {number_of_milestone_timebased}")
df_milestone_timebased = df[(df['chosen_feedback'] == 'Milestone feedback') & (df['vote_change'] == 'Timebased feedback')]
vote_change_explain_milestone_timebased = '\n'.join(df_milestone_timebased['vote_change_explain'].astype(str))
print(f"Some key reasons of participants choosing milestone feedback but changing to timebased feedback are: \n\n {summarise_text(vote_change_explain_milestone_timebased)}")

The number of participants who chose 'Milestone feedback' originally and changed to 'Timebased feedback' is 13
Some key reasons of participants choosing milestone feedback but changing to timebased feedback are: 

 - **desire for transparency**: many responses express a need for greater transparency in contributions to prevent imbalance.
- **mistrust and frustration**: contributors feel frustrated and hurt by selfish behavior when contributions aren't fairly distributed.
- **preference for option 2**: some respondents favor option 2 for its directness and less susceptibility to selfish actions.
- **impact on contribution decisions**: knowing team contributions would influence individual contribution decisions and potentially increase fairness.
- **calculating earnings**: participants want better methods to estimate their earnings based on contributions.


Remember, there is one participant who chose 'Milestone feedback' but forgot the choice.